# The Experiment Container

This notebook explains how the database works as an experiment container.

In [1]:
import qcodes as qc
from qcodes import initialise_database

## The database

The database on file is selected via the QCoDeS configuration file and a connection to it is set up at import time. There is always only one database selected as the "active" one.


### The config file

We'll be modifying the config file in a second, so first a brief word about the config system.

Only one config file can be used at a time. The default config file should not be overwritten, so if you have any modifications, you should save the updated config file on your home directory **OR** in the current working directory of your script/notebook. The QCoDeS config system first looks in the current directory for a config file and then in the home directory for one and only then - if no config files where found - does it fall back to using the default one. The default config is located in `qcodes/qcodes/config`.

In [2]:
# To inspect the config file, use
configuration = qc.config
print(f'Using config file from {configuration.current_config_path}')
print(f'Database location: {configuration["core"]["db_location"]}')

Using config file from /Users/william/sourcecodes/QCoDeS/qcodes/config/qcodesrc_schema.json
Database location: ./experiments.db


Note that the default setting for the database location is a relative path. That is a sane default setting, but not always what you want. If you execute scripts from many different folders on your machine, you should change the default location to an absolute path pointing to your data folder, or you will end up with multiple databases.

In [3]:
# Changing the settings can done like so:

configuration['core']['db_location'] = '~/my/particular/absolute/path/database.db'

# To keep the changes after a kernel restart, save them to disk
# configuration.save_to_home()  # this saves to the configuration file to the home directory

 For the sake of pedagogical clarity, we now create a new empty default database.

In [4]:
# point the config system to the new location
configuration['core']['db_location'] = './exp_container_tutorial.db'
# now save the config file in home. 
# NOTE: This config file will 
# from now on loaded by qcodes instead of the default.
configuration.save_to_home()
# create an empty database based on the config file
initialise_database()

## The experiments inside the database

The database holds a certain number of **experiments**. They may be viewed:

In [6]:
from qcodes import experiments

In [7]:
experiments()

[]

Not surprisingly, our new database is empty. Let us add some experiments.

In [8]:
from qcodes import new_experiment

In [9]:
new_experiment('first_exp', sample_name='old_sample')
new_experiment('second_exp', sample_name='slightly_newer_sample')
new_experiment('second_exp', sample_name='brand_new_sample')

second_exp#brand_new_sample#3@./exp_container_tutorial.db
---------------------------------------------------------

In [10]:
experiments()

[first_exp#old_sample#1@./exp_container_tutorial.db
 --------------------------------------------------,
 second_exp#slightly_newer_sample#2@./exp_container_tutorial.db
 --------------------------------------------------------------,
 second_exp#brand_new_sample#3@./exp_container_tutorial.db
 ---------------------------------------------------------]

We notice that each experiment is labelled by an integer. This is the **exp_id** that can be used when looking up properties of each experiment.

Let us add some runs to experiment 2 ("second_exp"). For the sake of clarity, we don't add any data to the runs here. We refer to the `Dataset Context Manager.ipynb` for detailed information on how to properly create and populate runs.

In [11]:
from qcodes import new_data_set

In [12]:
new_data_set('a_good_run', exp_id=2)

a_good_run #1@./exp_container_tutorial.db
-----------------------------------------

In [13]:
new_data_set('a_nother_run', exp_id=2)

a_nother_run #2@./exp_container_tutorial.db
-------------------------------------------

We may now inspect experiment 2.

In [14]:
from qcodes import load_experiment

In [15]:
exp_2 = load_experiment(2)

In [16]:
# Printing the experiment will reveal the runs

print(exp_2)

second_exp#slightly_newer_sample#2@./exp_container_tutorial.db
--------------------------------------------------------------
1-a_good_run-1--0
2-a_nother_run-2--0


In a similar way, we may of course add runs to the other two experiments. Now, the alert reader will have noticed that `exp_id` is a keyword argument in `new_data_set`. What is the default then? --The default is that the run is added to the experiment with the **highest** `exp_id` in the database.